In [2]:

import os

In [3]:

%pwd

'd:\\Projects\\Text_Summarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Projects\\Text_Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [23]:
os.listdir()

['.git',
 '.github',
 '.gitignore',
 'app.py',
 'config',
 'Dockerfile',
 'LICENSE',
 'main.py',
 'params.yaml',
 'README.md',
 'requirements.txt',
 'research',
 'setup.py',
 'src',
 'template.py',
 'venv']

In [27]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [63]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(

            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config

In [64]:
import os 
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [73]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} within the following info: {header} \n from {self.config.source_url}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extraxts the zip file into the data Directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [77]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-03-28 21:36:36,289:  INFO: yaml file loaded successfully: config\config.yaml]
[2025-03-28 21:36:36,292:  INFO: yaml file loaded successfully: params.yaml]
[2025-03-28 21:36:36,295:  INFO: Directory created at artifacts]
[2025-03-28 21:36:36,297:  INFO: Directory created at artifacts/data_ingestion]


FileNotFoundError: [Errno 2] No such file or directory: 'data/ingestion/data.zip'